In [37]:
import pandas as pd
import os
import json as js
from pandas.io.json import json_normalize
from datetime import datetime, timedelta

In [38]:
path = './sokulee'
dir_list = os.listdir(path)
print(dir_list)

['A01', 'A010', 'A016', 'A017', 'A018', 'A019', 'A02', 'A020', 'A021', 'A022', 'A024', 'A025', 'A026', 'A027', 'A028', 'A029', 'A03', 'A030', 'A031', 'A032', 'A033', 'A034', 'A035', 'A036', 'A037', 'A038', 'A039', 'A04', 'A040', 'A041', 'A042', 'A043', 'A044', 'A045', 'A047', 'A048', 'A049', 'A05', 'A050', 'A052', 'A053', 'A054', 'A056', 'A057', 'A058', 'A059', 'A06', 'A060', 'A061', 'A062', 'A063', 'A065', 'A068', 'A07', 'A071', 'A072', 'A073', 'A08', 'A080', 'A081', 'A083', 'A084', 'A092', 'A093', 'A094', 'A096', 'A097', 'A098']


In [39]:
df_heart = pd.DataFrame(columns=['USERNAME', 'DATE', 'TIME', 'VALUE'])
df_sleep = pd.DataFrame(columns=['USERNAME', 'DATE', 'STARTTIME', 'MINUTESASLEEP', 'MINUTUESAWAKE', 'TIMEINBED'])
df_step = pd.DataFrame(columns=['USERNAME', 'DATE', 'TIME', 'HOUR', 'MINUTES', 'STEPS'])

In [40]:
for i in range(len(dir_list)):
    file_list = os.listdir('./sokulee/'+dir_list[i])
    user_name = dir_list[i]
    
    print(dir_list[i])
    #print(len(user_name))
    for j in range(len(file_list)):
        file_path = './sokulee/'+dir_list[i]+'/'+file_list[j]
        file = open(file_path)
        data = js.load(file)
        #print(file_list[j])
        
        if len(user_name) == 3:
            date = file_list[j][4:8]+"-"+file_list[j][8:10]+"-"+file_list[j][10:12]
        elif len(user_name) > 3:
            date = file_list[j][5:9]+"-"+file_list[j][9:11]+"-"+file_list[j][11:13]
            
            
        if 'heart' in file_path:
            if len(pd.DataFrame(data["activities-heart-intraday"]["dataset"])) > 0:
                tmp_df = pd.DataFrame(data["activities-heart-intraday"]["dataset"])
            
                tmp_df['USERNAME'] = user_name
                tmp_df['DATE'] = date
            
                tmp_df = pd.DataFrame(tmp_df, columns=['USERNAME', 'DATE', 'time', 'value'])
                tmp_df.rename(columns={'time' : 'TIME', 'value' : 'VALUE'}, inplace=True)
        
                df_heart = pd.concat([df_heart, tmp_df])
            
            else:
                tmp_df = pd.DataFrame(columns=['USERNAME', 'DATE', 'TIME', 'VALUE'])
                tmp_df.loc[0] = [user_name, date, "00:00:00", 0]
                df_heart = pd.concat([df_heart, tmp_df])
                
                
        elif 'sleep' in file_path:
            if len(pd.DataFrame(data['sleep'])) > 0:
                tmp_df  = pd.DataFrame(data["sleep"][0])
           
                start_time = tmp_df['startTime'][0][11:19]
                time_in_bed = tmp_df['timeInBed'][0]
            
                asleep = tmp_df['minutesAsleep'][0]
                awake = tmp_df['minutesAwake'][0]
            
            
                tmp_df = pd.DataFrame(columns=['USERNAME', 'DATE', 'STARTTIME', 'MINUTESASLEEP', 'MINUTUESAWAKE', 'TIMEINBED'])
                tmp_df.loc[0] = [user_name, date, start_time, asleep, awake, time_in_bed]
                df_sleep = pd.concat([df_sleep, tmp_df])
                
        
        
        elif 'steps' in file_path:
            if len(pd.DataFrame(data["activities-steps-intraday"]["dataset"])) > 0:
                tmp_df = pd.DataFrame(data["activities-steps-intraday"]["dataset"])
            
                tmp_df['USERNAME'] = user_name
                tmp_df['DATE'] = date
                tmp_df['HOUR'] = tmp_df['time'].apply(lambda e: int(e[0:2]))
                tmp_df['MINUTES'] = tmp_df['time'].apply(lambda e: int(e[3:5]))
            
                tmp_df = pd.DataFrame(tmp_df, columns=['USERNAME', 'DATE', 'time', 'HOUR', 'MINUTES', 'value'])
                tmp_df.rename(columns={'time' : 'TIME', 'value' : 'STEPS'}, inplace=True)
        
                df_step = pd.concat([df_step, tmp_df])
                #print(df_step)
                
            else:
                tmp_df = pd.DataFrame(columns=['USERNAME', 'DATE', 'TIME', 'HOUR', 'MINUTES', 'STEPS'])
                tmp_df.loc[0] = [user_name, date, "00:00:00", "0", "0", "0"]
                df_step = pd.concat([df_step, tmp_df])

A01
A010
A016
A017
A018
A019
A02
A020
A021
A022
A024
A025
A026
A027
A028
A029
A03
A030
A031
A032
A033
A034
A035
A036
A037
A038
A039
A04
A040
A041
A042
A043
A044
A045
A047
A048
A049
A05
A050
A052
A053
A054
A056
A057
A058
A059
A06
A060
A061
A062
A063
A065
A068
A07
A071
A072
A073
A08
A080
A081
A083
A084
A092
A093
A094
A096
A097
A098


In [41]:
df_heart.to_csv('./all_user_heart.csv', index=False)
df_sleep.to_csv('./all_user_sleep.csv', index=False)
df_step.to_csv('./all_user_steps.csv', index=False)